In [1]:
import pandas as pds
from astropy.time import Time
from datetime import timedelta

# Get official leap seconds list from International Time Bureau

In [2]:
leap_seconds = pds.read_csv("https://www.ietf.org/timezones/data/leap-seconds.list", comment="#", sep='\t', names=["epoch","offset"], index_col=False) 

# Print leap seconds list 

In [3]:
for sec in leap_seconds.values: 
    print(Time("1900-01-01") + timedelta(seconds=float(sec[0]+sec[1]))) 

1972-01-01 00:00:00.000
1972-07-01 00:00:00.000
1973-01-01 00:00:00.000
1974-01-01 00:00:00.000
1975-01-01 00:00:00.000
1976-01-01 00:00:00.000
1977-01-01 00:00:00.000
1978-01-01 00:00:00.000
1979-01-01 00:00:00.000
1980-01-01 00:00:00.000
1981-07-01 00:00:00.000
1982-07-01 00:00:00.000
1983-07-01 00:00:00.000
1985-07-01 00:00:00.000
1988-01-01 00:00:00.000
1990-01-01 00:00:00.000
1991-01-01 00:00:00.000
1992-07-01 00:00:00.000
1993-07-01 00:00:00.000
1994-07-01 00:00:00.000
1996-01-01 00:00:00.000
1997-07-01 00:00:00.000
1999-01-01 00:00:00.000
2006-01-01 00:00:00.000
2009-01-01 00:00:00.000
2012-07-01 00:00:00.000
2015-07-01 00:00:00.000
2017-01-01 00:00:00.000


# generate C++ code for CDF++ lib

In [6]:
print("""constexpr std::array leap_seconds_J2000 = 
{""")
for sec in leap_seconds.values: 
    date = str(Time("1900-01-01") + timedelta(seconds=float(sec[0]+sec[1]))).split()[0].split('-')
    print(f"    std::pair{{ duration_cast<seconds>( (sys_days {{ {date[0]}_y / {date[1]} / {date[2]} }} + 0h ) - _J2000).count() , {sec[1]} - 32 }},") 
print("""};""")


constexpr std::array leap_seconds_J2000 = 
{
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 01 / 01 } + 0h ) - _J2000).count() , 10 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 07 / 01 } + 0h ) - _J2000).count() , 11 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1973_y / 01 / 01 } + 0h ) - _J2000).count() , 12 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1974_y / 01 / 01 } + 0h ) - _J2000).count() , 13 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1975_y / 01 / 01 } + 0h ) - _J2000).count() , 14 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1976_y / 01 / 01 } + 0h ) - _J2000).count() , 15 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1977_y / 01 / 01 } + 0h ) - _J2000).count() , 16 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1978_y / 01 / 01 } + 0h ) - _J2000).count() , 17 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1979_y / 01 / 01 } + 0h ) - _J2000).coun

In [5]:
print("""constexpr std::array leap_seconds_J2000_reverse = 
{""")
for sec in leap_seconds.values: 
    date = str(Time("1900-01-01") + timedelta(seconds=float(sec[0]+sec[1]))).split()[0].split('-')
    print(f"    std::pair{{ duration_cast<seconds>( (sys_days {{ {date[0]}_y / {date[1]} / {date[2]} }} + 0h ) - _J2000).count() + {sec[1]} - 32 , {sec[1]} - 32 }},") 
print("""};""")


constexpr std::array leap_seconds_J2000_reverse = 
{
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 01 / 01 } + 0h ) - _J2000).count() + 10 - 32 , 10 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1972_y / 07 / 01 } + 0h ) - _J2000).count() + 11 - 32 , 11 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1973_y / 01 / 01 } + 0h ) - _J2000).count() + 12 - 32 , 12 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1974_y / 01 / 01 } + 0h ) - _J2000).count() + 13 - 32 , 13 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1975_y / 01 / 01 } + 0h ) - _J2000).count() + 14 - 32 , 14 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1976_y / 01 / 01 } + 0h ) - _J2000).count() + 15 - 32 , 15 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1977_y / 01 / 01 } + 0h ) - _J2000).count() + 16 - 32 , 16 - 32 },
    std::pair{ duration_cast<seconds>( (sys_days { 1978_y / 01 / 01 } + 0h ) - _J2000).count() + 17 - 32 , 17 - 32 },
   